# 1. Importación de paquetes

In [18]:
#Import packages
from modules import preprocess
from modules import training
import pandas as pd

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [19]:
df = preprocess.load_csv_data('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [20]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [21]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [22]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [23]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *train* y *test*

In [24]:
# Split dataset into train and test
train_df, test_df = preprocess.split_dataset(df)
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [25]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
5637,25.25,25.25,25.00,25.81,17,20,75,53,0.12,0.09,0.19,0.10,355,0.000000,0,0,0
5844,25.38,25.38,25.25,26.00,18,22,79,57,0.07,0.05,0.06,0.09,355,0.000000,0,0,0
4763,25.25,25.25,24.69,25.63,0,0,0,0,0.06,0.05,0.06,0.06,360,0.000000,0,0,0
4226,25.44,25.44,24.94,25.56,0,0,0,0,0.07,0.05,0.07,0.07,370,-0.119231,0,0,0
4994,25.13,25.19,24.56,25.44,0,0,0,0,0.07,0.05,0.06,0.06,360,-0.011538,0,0,0


In [26]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
1413,25.44,25.44,25.00,25.81,0,0,0,0,0.07,0.05,0.06,0.06,385,0.000000,0,0,0
8834,25.38,25.38,25.00,25.50,0,0,0,0,0.06,0.05,0.05,0.09,360,-0.238462,0,0,0
7469,25.25,25.31,24.81,25.63,0,0,0,0,0.08,0.04,0.07,0.10,355,0.000000,0,0,0
2092,25.13,25.13,24.56,25.44,0,0,0,0,0.08,0.05,0.06,0.05,360,0.000000,0,0,0
5511,25.19,25.13,24.81,25.75,12,14,56,40,0.08,0.04,0.08,0.09,355,0.000000,0,0,0


## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [27]:
# Get features columns
X = preprocess.get_features(train_df)

# Get output column
y = preprocess.get_output(train_df)

In [28]:
X.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
5637,25.25,25.25,25.00,25.81,17,20,75,53,0.12,0.09,0.19,0.10,355,0.000000,0,0
5844,25.38,25.38,25.25,26.00,18,22,79,57,0.07,0.05,0.06,0.09,355,0.000000,0,0
4763,25.25,25.25,24.69,25.63,0,0,0,0,0.06,0.05,0.06,0.06,360,0.000000,0,0
4226,25.44,25.44,24.94,25.56,0,0,0,0,0.07,0.05,0.07,0.07,370,-0.119231,0,0
4994,25.13,25.19,24.56,25.44,0,0,0,0,0.07,0.05,0.06,0.06,360,-0.011538,0,0


In [29]:
y.head()

5637    0
5844    0
4763    0
4226    0
4994    0
Name: Room_Occupancy_Count, dtype: int64

# 2. Entrenamiento y seguimiento de la huella de carbono con **CodeCarbon**

In [30]:
training.train_LR_eco2ai(df,0.25)

/home/misanchz/miniconda3/envs/TFG-Project/lib/python3.10/site-packages/eco2ai/emission_track.py:211: UserWarning: 
    If you use a VPN, you may have problems with identifying your country by IP.
    It is recommended to disable VPN or
    manually install the ISO-Alpha-2 code of your country during initialization of the Tracker() class.
    You can find the ISO-Alpha-2 code of your country here: https://www.iban.com/country-codes
    
  warnings.warn(
/home/misanchz/miniconda3/envs/TFG-Project/lib/python3.10/site-packages/eco2ai/tools/tools_gpu.py:35: NoGPUWarning: 

There is no any available GPU devices or your gpu is not supported by Nvidia library!
The thacker will consider CPU usage only

  warnings.warn(message="\n\nThere is no any available GPU devices or your gpu is not supported by Nvidia library!\nThe thacker will consider CPU usage only\n",


In [36]:
df_emissions = pd.read_csv("eco2ai_emissions.csv",sep=",")
df_emissions

,project_name,experiment_description(model type etc.),start_time,duration(s),power_consumption(kWTh),CO2_emissions(kg),CPU_name,GPU_name,OS,region/country
0,TFG_Project,training LogisticRegression model,2022-08-28 11:11:31,0.773022,5.531269e-09,1.103571e-09,Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz/1 dev...,0 device(s),Linux,ES/Aragon
1,TFG_Project,training LogisticRegression model,2022-08-28 11:12:47,0.264309,4.498689e-08,8.975559e-09,Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz/1 dev...,0 device(s),Linux,ES/Aragon


In [32]:
# Random Forest
#training.train_RF_codecarbon(X,y)

In [33]:
# Linear SVC (Support Vector Machines)
#training.train_SVC_codecarbon(X,y)